# Consolidating data

This file consists of functions that consolidate our disparate datasets into one large dataset that is useful in training our model. 

The goal is to generate a file with 30 columns (this number should be variable), such that each column is a state in time. 

Ideally, this will be done with heirachical data, ie `p1` is the first point in time, and within `p1` you have an x component, y component, etc.

https://pandas.pydata.org/docs/user_guide/advanced.html

## Input data format

It is assumed that the input data with have the columns: `[timestamp,tx,ty,tz,qx,qy,qz,qw]`

## Extracting the data we want

This function will create velocity and acceleration columns.

In [24]:
import numpy as np
import pandas as pd

def extract_features(raw: pd.DataFrame, dropna: bool = False) -> None:
    raw['vx'] = raw['tx'].diff() / raw['timestamp'].diff()
    raw['vy'] = raw['ty'].diff() / raw['timestamp'].diff()
    raw['vz'] = raw['tz'].diff() / raw['timestamp'].diff()

    raw['ax'] = raw['vx'].diff() / raw['timestamp'].diff()
    raw['ay'] = raw['vy'].diff() / raw['timestamp'].diff()
    raw['az'] = raw['vz'].diff() / raw['timestamp'].diff()

    if dropna: raw.dropna(inplace=True)

In [26]:
# test the above functions

df = pd.read_csv("../data/fpv_uzh/indoor_forward_3_davis_with_gt.txt")

extract_features(df, dropna=True)

print(df.head())
print(df['timestamp'])

      timestamp        tx        ty        tz        qx        qy        qz  \
2  1.540820e+09  7.603792  0.242395 -0.753890 -0.269081 -0.661483  0.642058   
3  1.540820e+09  7.603930  0.243608 -0.753434 -0.269001 -0.661389  0.642086   
4  1.540820e+09  7.604787  0.244973 -0.752511 -0.268968 -0.661503  0.641952   
5  1.540820e+09  7.605305  0.246114 -0.751615 -0.269007 -0.661649  0.641821   
6  1.540820e+09  7.605622  0.246968 -0.751001 -0.268870 -0.661571  0.641922   

         qw        vx        vy        vz        ax        ay        az  
2  0.278923 -0.006013  0.009916  0.003732  0.048684 -0.009624  0.023827  
3  0.279157  0.001377  0.012131  0.004557  0.073898  0.022147  0.008241  
4  0.279228  0.008571  0.013648  0.009230  0.071941  0.015168  0.046731  
5  0.279146  0.005177  0.011415  0.008960 -0.033934 -0.022326 -0.002695  
6  0.279229  0.003167  0.008536  0.006144 -0.020107 -0.028794 -0.028158  
2      1.540820e+09
3      1.540820e+09
4      1.540820e+09
5      1.540820e+09
6

In [61]:
trial = df[0:4]
trial.head()

,timestamp,tx,ty,tz,qx,qy,qz,qw,vx,vy,vz,ax,ay,az
2,1.540820e+09,7.603792,0.242395,-0.753890,-0.269081,-0.661483,0.642058,0.278923,-0.006013,0.009916,0.003732,0.048684,-0.009624,0.023827
3,1.540820e+09,7.603930,0.243608,-0.753434,-0.269001,-0.661389,0.642086,0.279157,0.001377,0.012131,0.004557,0.073898,0.022147,0.008241
4,1.540820e+09,7.604787,0.244973,-0.752511,-0.268968,-0.661503,0.641952,0.279228,0.008571,0.013648,0.009230,0.071941,0.015168,0.046731
5,1.540820e+09,7.605305,0.246114,-0.751615,-0.269007,-0.661649,0.641821,0.279146,0.005177,0.011415,0.008960,-0.033934,-0.022326,-0.002695


In [65]:
# temp = np.DataFrame.arange(4)
# print(temp)
trial.loc[:, "trajNum"] = np.arange(len(trial))
trial.loc[:, "slice"] = 1
trial.head()

C:\Users\megan\AppData\Local\Temp\ipykernel_37320\1633099997.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial.loc[:, "slice"] = 1


,timestamp,tx,ty,tz,qx,qy,qz,qw,vx,vy,vz,ax,ay,az,trajNum,slice
2,1.540820e+09,7.603792,0.242395,-0.753890,-0.269081,-0.661483,0.642058,0.278923,-0.006013,0.009916,0.003732,0.048684,-0.009624,0.023827,0,1
3,1.540820e+09,7.603930,0.243608,-0.753434,-0.269001,-0.661389,0.642086,0.279157,0.001377,0.012131,0.004557,0.073898,0.022147,0.008241,1,1
4,1.540820e+09,7.604787,0.244973,-0.752511,-0.268968,-0.661503,0.641952,0.279228,0.008571,0.013648,0.009230,0.071941,0.015168,0.046731,2,1
5,1.540820e+09,7.605305,0.246114,-0.751615,-0.269007,-0.661649,0.641821,0.279146,0.005177,0.011415,0.008960,-0.033934,-0.022326,-0.002695,3,1


In [69]:
trial.set_index(['slice', 'trajNum'], inplace=True)
trial.head()

timestamp        tx        ty        tz        qx        qy  \
slice trajNum                                                                   
1     0        1.540820e+09  7.603792  0.242395 -0.753890 -0.269081 -0.661483   
      1        1.540820e+09  7.603930  0.243608 -0.753434 -0.269001 -0.661389   
      2        1.540820e+09  7.604787  0.244973 -0.752511 -0.268968 -0.661503   
      3        1.540820e+09  7.605305  0.246114 -0.751615 -0.269007 -0.661649   

                     qz        qw        vx        vy        vz        ax  \
slice trajNum                                                               
1     0        0.642058  0.278923 -0.006013  0.009916  0.003732  0.048684   
      1        0.642086  0.279157  0.001377  0.012131  0.004557  0.073898   
      2        0.641952  0.279228  0.008571  0.013648  0.009230  0.071941   
      3        0.641821  0.279146  0.005177  0.011415  0.008960 -0.033934   

                     ay        az  
slice trajNum                      
1     0       -0.009624  0.023827  
      1        0.022147  0.008241  
      2        0.015168  0.046731  
      3       -0.022326 -0.002695

## Slicing the data

Now, we want rows of data that represent a specific range of time. In this case, we want 30 points for each new row.

In [18]:
def generate_slices(data: pd.DataFrame, n: int) -> pd.DataFrame:
    # each row in the original data is a "point". Each row in the output 
    # is a list of points of size n. 
    cols = [f"{col}_{i}" for i in range(n) for col in data.columns]
    slices = []
    for i in range(len(data) - n):
        flattened = pd.DataFrame([data[i:i+n].to_numpy().flatten()])
        flattened.columns = cols
        slices.append(flattened)
    return pd.concat(slices, ignore_index=False)

In [20]:
# test the above function for 4 pints in each row

slices = generate_slices(df, 4)
# print(slices.head())
slices.head()

,timestamp_0,tx_0,ty_0,tz_0,qx_0,qy_0,qz_0,qw_0,vx_0,vy_0,...,qx_3,qy_3,qz_3,qw_3,vx_3,vy_3,vz_3,ax_3,ay_3,az_3
0,1.540820e+09,7.603792,0.242395,-0.753890,-0.269081,-0.661483,0.642058,0.278923,-0.006013,0.009916,...,-0.269007,-0.661649,0.641821,0.279146,0.005177,0.011415,0.008960,-0.033934,-0.022326,-0.002695
0,1.540820e+09,7.603930,0.243608,-0.753434,-0.269001,-0.661389,0.642086,0.279157,0.001377,0.012131,...,-0.268870,-0.661571,0.641922,0.279229,0.003167,0.008536,0.006144,-0.020107,-0.028794,-0.028158
0,1.540820e+09,7.604787,0.244973,-0.752511,-0.268968,-0.661503,0.641952,0.279228,0.008571,0.013648,...,-0.269057,-0.662028,0.641452,0.279046,0.008559,0.006565,-0.007456,0.053924,-0.019712,-0.136007
0,1.540820e+09,7.605305,0.246114,-0.751615,-0.269007,-0.661649,0.641821,0.279146,0.005177,0.011415,...,-0.269372,-0.662669,0.640559,0.279272,-0.000637,-0.017040,-0.007900,-0.091958,-0.236045,-0.004439
0,1.540820e+09,7.605622,0.246968,-0.751001,-0.268870,-0.661571,0.641922,0.279229,0.003167,0.008536,...,-0.269935,-0.662853,0.640029,0.279507,-0.001469,-0.013451,-0.002426,-0.008326,0.035883,0.054737


In [123]:
def multiIndex(data: pd.DataFrame, n: int, j: int) -> pd.DataFrame:
    # each row in the original data is a "point". Each row in the output 
    # is a list of points of size n. 
    cols = [f"{col}_{i}" for i in range(n) for col in data.columns]
    slices = []
    # i is the number of slices we want
    for i in range(len(data) - n):
        trial = data.copy()[i:i+n]
        trial.loc[:, "trajNum"] = np.arange(len(trial))
        trial.loc[:, "slice"] = str(j) + ":" + str(i)
        trial.set_index(['slice', 'trajNum'], inplace=True)
        slices.append(trial)
    return pd.concat(slices, ignore_index=False)

In [125]:
slices = multiIndex(df, 10, 0)
# print(slices.head())
print(slices.head(20))
type(slices)

               timestamp       tx       ty       tz      vx      vy      vz  \
slice trajNum                                                                 
0:0   0              0.0  0.08312  0.46529 -0.56110     NaN     NaN     NaN   
      1              0.1  0.10778  0.49733 -0.52509  0.2466  0.3204  0.3601   
      2              0.2  0.14227  0.51815 -0.47645  0.3449  0.2082  0.4864   
      3              0.3  0.15770  0.52431 -0.42075  0.1543  0.0616  0.5570   
      4              0.4  0.18045  0.52961 -0.36707  0.2275  0.0530  0.5368   
      5              0.5  0.19404  0.50781 -0.31943  0.1359 -0.2180  0.4764   
      6              0.6  0.20461  0.47123 -0.24595  0.1057 -0.3658  0.7348   
      7              0.7  0.21739  0.44461 -0.22270  0.1278 -0.2662  0.2325   
      8              0.8  0.20570  0.41468 -0.16866 -0.1169 -0.2993  0.5404   
      9              0.9  0.17935  0.37803 -0.14353 -0.2635 -0.3665  0.2513   
0:1   0              0.1  0.10778  0.49733 -0.52509 

pandas.core.frame.DataFrame

## Consolidate all our original data

Now, we want to consolidate our data from all the other sources.

In [22]:
import os

fpv_data = "../data/fpv_uzh"
random_traj_data = "../data/random_trajectory_100ms"
output_path = "../data/output"
if not os.path.exists(output_path):
    os.makedirs(output_path)

n = 30 # we want 30 points per row
slices = []

# consolidate the fpv data
for filename in filter(lambda p: p.endswith("txt"), os.listdir(fpv_data)):
    filepath = os.path.join(fpv_data, filename)
    df = pd.read_csv(filepath)
    
    extract_features(df)
    # the columns should be handled in the generate slices funciton
    slices.append(generate_slices(df, n))
    
# consolidate the synthetic data
for filename in filter(lambda p: p.endswith("txt"), os.listdir(random_traj_data)):
    filepath = os.path.join(random_traj_data, filename)
    df = pd.read_csv(filepath)
    
    extract_features(df)
    # the columns should be handled in the generate slices funciton
    slices.append(generate_slices(df, n))

consolidated = pd.concat(slices, ignore_index=False)
# consolidated.to_csv(os.path.join(output_path, "consolidated.csv"))

In [127]:
import os

# THIS IS FOR THE MULTI-INDEXING
fpv_data = "../data/fpv_uzh"
random_traj_data = "../data/random_trajectory_100ms"
output_path = "../data/output"
if not os.path.exists(output_path):
    os.makedirs(output_path)

n = 30 # we want 30 points per row
slices = []
j = 0 # hold the slice number for the entire consolidation for the multi-indexing

# consolidate the fpv data
for filename in filter(lambda p: p.endswith("txt"), os.listdir(fpv_data)):
    filepath = os.path.join(fpv_data, filename)
    df = pd.read_csv(filepath)
    
    extract_features(df)
    # the columns should be handled in the generate slices funciton
    slices.append(multiIndex(df, n, j))
    j += 1
    
# consolidate the synthetic data
for filename in filter(lambda p: p.endswith("txt"), os.listdir(random_traj_data)):
    filepath = os.path.join(random_traj_data, filename)
    df = pd.read_csv(filepath)
    
    extract_features(df)
    # the columns should be handled in the generate slices funciton
    slices.append(multiIndex(df, n, j))
    j += 1

consolidated = pd.concat(slices, ignore_index=False)
consolidated.to_csv(os.path.join(output_path, "consolidated.csv"))